In [296]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from  tqdm import tqdm
import time
import re
import csv

## 1.1 List of Places

In [294]:
for i in tqdm(range(1,401)):
    
    #We change along the first 400 pages and request via their urls
    url='https://www.atlasobscura.com/places?page='+str(i)+'&sort=likes_count'
    result=requests.get(url)
    soup=BeautifulSoup(result.text,'html.parser')
    
    try:
        #Then we find within the same page, all the posts that have this specific class
        aux=soup.find_all('div',{'class':"col-md-4 col-sm-6 col-xs-12"})
        
        #Later we save in list_places the urls of each of these posts
        list_places=['https://www.atlasobscura.com'+aux[x].find('a').get('href') for x in range(len(aux))]
        
        #To conclude, we save it in a .txt file with all the 
        with open('urls.txt','a') as file:
            for j in range(len(list_places)):
                file.write(list_places[j])
                file.write('\n')
    except:
        pass
    # This will leave a second between each iteration so we do not get banned from the website
    time.sleep(1) 
    

100%|█████████████████████████████████████████| 400/400 [35:52<00:00,  5.38s/it]


## 1.2 Crawl Places

In [297]:
#We first read the lines of the file with the urls
archive=open('urls.txt')
urls=archive.readlines()

#We do this so we only need to acces the webpage once to get all the data and not enter it 18*400 times

for i in tqdm(range(len(urls))):
    #We only need the urls, not the '\n'
    link=urls[i].strip()
    #we get the html within the same session
    html=requests.get(link)
    #and write it in differents files
    with open('Html-'+str(i+1)+'.txt','a') as doc:
        doc.write(html.text)
    time.sleep(1)
    

100%|█████████████████████████████████████| 7362/7362 [4:02:14<00:00,  1.97s/it]


## 1.3 Parse Downloaded pages

Here we are goin to implement a function to extract all the data we need from each html and then we will put it together in a csv file.

For each variable required we will use try and except and if the variable is empty or there is some problem with it, it will be left in blank, with an empty space.

In [303]:
def get_info(html):
    
    info={}
    # We want
    # Place Name (to save as placeName): String.
    try:
        info['placeName']=html.find('h1',{'class':"DDPage__header-title"}).text
    except:
        info['placeName']=' '
    
    # Place Tags (to save as placeTags): List of Strings.
    try:
        p=html.find('div',{'class':"DDPage__header-place-info"})
        p=p.find('div',{'class':"DDPage__header-place-location"})
        p=p.find('a').text.split(',')
        info['placeTags']=','.join(p)
    except:
        info['placeTags']=[' ']
        
    # Number of people who have been there (to save as numPeopleVisited): Integer.
    try:
        ppl=html.find('div',{'class':"col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print"})
        ppl=ppl.find('div',{'class':'title-md item-action-count'})
        info['numPeopleVisited']=int(ppl.text)
    except:
        info['numPeopleVisited']=' '
        
    # Number of people who want to visit the place(to save as numPeopleWant): Integer.
    try:
        ppl=html.find('div',{'class':"col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print"})
        ppl=ppl.find('div',{'class':'title-md item-action-count'})
        info['numPeopleWant']=int(ppl.text)
    except:
        info['numPeopleWant']=' '
    
    # Description (to save as placeDesc): String. Everything from under the first image up to "know before you go" (orange frame on the example image).
    try:
        des=html.find('div',{'class':'DDPageContent__column grid-col-lg-6 grid-col-md-7'})
        des=des.find('div',{'class':'DDP__body-copy'})
        des=des.find_all('p')
        aux=''
        for i in range(len(des)):
            aux+=str(des[i].text)
            
        info['placeDesc']=aux
    except:
        info['placeDesc']=' '

        
    # Short Description (to save as placeShortDesc): String. Everything from the title and location up to the image (blue frame on the example image).
    
    placeShortDesc=" "
    try:
        placeShortDesc=html.find("h3", {"class","DDPage__header-dek"}).text.strip()
    except:
        placeShortDesc=" "
    info['placeShortDesc']=placeShortDesc
    
    
    # Nearby Places (to save as placeNearby): Extract the names of all nearby places, but only keep unique values: List of Strings.
    
    placeNearby=[]
    
    try:
        near=html.find("div",{"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"})
        near=near.find('div',{'class':'DDPageSiderailRecirc'})
        near=near.find_all('a')
        for j in range(len(near)):
            aux=near[j].find('div',{'class':'DDPageSiderailRecirc__item-title'})
            placeNearby.append(aux.text.strip())
    except:
        pass
            
    info['planceNearby']=','.join(list(set(placeNearby)))
    
    # Address of the place(to save as placeAddress): String.
    
    try:
        place=html.find('div',{'class':'DDPageSiderail__column grid-col-lg-4 grid-col-md-5'})
        place=place.find('div',{'class':'DDPageSiderail'})
        place=place.find('aside', {'class':'DDPageSiderail__details'})
        place=place.find('address',{'class':'DDPageSiderail__address'})
        place=place.find('div').get_text(' , ').strip().rstrip(' , ').strip()
    except:
        place=' '
    info['placeAdress']=place.strip()
    
    # Altitude and Longitude of the place's location(to save as placeAlt and placeLong): Integers
    try:
        coord=html.find('div',{'class':'DDPageSiderail__column grid-col-lg-4 grid-col-md-5'})
        coord=coord.find('div',{'class':'DDPageSiderail'})
        coord=coord.find('div',{'class':'DDPageSiderail__coordinates js-copy-coordinates'}).text.strip().split(',')
        x,y=coord[0],coord[1]
    except:
        x,y=' ',' '
    info['placeAlt']=float(x)
    info['placeLong']=float(y)
    
    # The username of the post editors (to save as placeEditors): List of Strings.

    editors=[]
    edits=html.find('div',{'id':'ugc-module'})
    edits=edits.find('div',{'class':'DDPContributors'})
    edits=edits.find_all('div',{'class':'ugc-editors'})
    try:
        for i in range(len(edits)):
            #For each editor, we look if it added the post or edited it because they have different structures
            aux=edits[i].find('h6').text.strip()
            if aux.lower() == 'added by':
                try:
                    add=edits[i].find("div",{"class":"DDPContributorsList"})
                    editors.append(add.find("a").text)
                except:
                    add=' '
            if aux.lower()== 'edited by':
                try:
                    add=edits[i].find("div",{"class":"DDPContributorsList"})
                    add=add.find("div",{"class":"js-editor-list hidden"})
                    add=add.find_all('li')
                    for j in range(len(add)):
                        name=add[j].find('span').text.strip()
                        editors.append(name)
                except:
                    add=' '
            
    except:
        editors=['']
    
    info['placeEditors']=','.join(editors)
    
    # Post publishing date (to save as placePubDate): datetime.
    try:
        t=html.find('div',{'id':'ugc-module'})
        t=t.find('div',{'class':'DDPContributor__name'}).text
        t=datetime.strptime(t, "%B %d, %Y").date()
    except:
        t=''
    info['placePubDate']=t
    
    # The names of the lists that the place was included in (to save as placeRelatedLists): List of Strings.
    
    included=[]
    try:
        related=html.find('div',{'class':'item-tags col-xs-12'})
        related=related.find_all('a')
        for i in range(len(related)):
            included.append(related[i].text.strip())
        
    except:
        related=''
    info['placeRelatedPlaces']=','.join(included)
    
    # The names of the related places (to save as placeRelatedPlaces): List of Strings.
    relatedp=[]
    try:
        places=html.find_all('div',{'class':'card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links'})
        aux=places[1].find_all('div',{'class':'CardWrapper --PlaceWrapper js-CardWrapper'})
        for i in range(len(aux)):
            p=aux[i].find('h3').text.strip()
            relatedp.append(p)
    except:
        pass
    info['placeRelatedPlaces']=','.join(relatedp)
    
    # The URL of the page of the place (to save as placeURL):String
    try:
        link=html.find('link',{'rel':'canonical'}).get('href')
    except:
        link=''
    info['placeURL']=link    
    
    return info

Now, we call the function for each of the htmls we have previouslt downloaded and keep the information of each in a `tsv` file.

In [310]:
for i in tqdm(range(len(urls))):
    with open('Html-'+str(i+1)+'.txt') as file:
        html = file.read()
    html=BeautifulSoup(html,'html.parser') 

    info=get_info(html)
    df=pd.DataFrame(info,index=[0])
    df.to_csv('place_'+str(i+1)+'.tsv',sep="\t")

100%|███████████████████████████████████████| 7362/7362 [22:09<00:00,  5.54it/s]


For the sake of simplicity of moving the data around, we will also put it all together in a `csv` file.

In [316]:
with open('Html-1.txt') as file:   # We put the first html and then append the others
    html = file.read()
    html=BeautifulSoup(html,'html.parser') 

    info=get_info(html)
    df=pd.DataFrame(info,index=[0])
    
for i in tqdm(range(2,len(urls))):
    
    with open('Html-'+str(i)+'.txt') as file:
        html = file.read()
        
    html=BeautifulSoup(html,'html.parser') 
    info=get_info(html)
    aux=pd.DataFrame(info,index=[i-1])
    df=df.append(aux)

df.to_csv('places.csv',sep='\t')

  0%|                                                  | 0/7360 [00:00<?, ?it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  0%|                                          | 1/7360 [00:00<35:03,  3.50it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  0%|                                        | 2/7360 [00:00<1:02:50,  1.95it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  0%|                                          | 3/7360

  0%|▏                                        | 26/7360 [00:07<29:08,  4.19it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  0%|▏                                        | 27/7360 [00:07<28:23,  4.31it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  0%|▏                                        | 28/7360 [00:07<28:29,  4.29it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  0%|▏                                        | 29/7360

  1%|▎                                        | 52/7360 [00:14<32:04,  3.80it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▎                                        | 53/7360 [00:14<30:54,  3.94it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▎                                        | 54/7360 [00:14<29:24,  4.14it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▎                                        | 55/7360

  1%|▍                                        | 78/7360 [00:21<40:54,  2.97it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▍                                        | 79/7360 [00:21<37:20,  3.25it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▍                                        | 80/7360 [00:22<34:50,  3.48it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▍                                        | 81/7360

  1%|▌                                       | 104/7360 [00:28<30:20,  3.99it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▌                                       | 105/7360 [00:28<29:48,  4.06it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▌                                       | 106/7360 [00:28<29:27,  4.10it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
  1%|▌                                       | 107/7360

...

 99%|██████████████████████████████████████▍| 7254/7360 [18:03<00:15,  7.06it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▍| 7255/7360 [18:04<00:14,  7.34it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▍| 7256/7360 [18:04<00:13,  7.53it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▍| 7257/7360

 99%|██████████████████████████████████████▌| 7280/7360 [18:07<00:10,  7.74it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▌| 7281/7360 [18:07<00:10,  7.74it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▌| 7282/7360 [18:07<00:09,  7.89it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▌| 7283/7360

 99%|██████████████████████████████████████▋| 7306/7360 [18:11<00:06,  7.89it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▋| 7307/7360 [18:11<00:06,  8.01it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▋| 7308/7360 [18:11<00:06,  8.06it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
 99%|██████████████████████████████████████▋| 7309/7360

100%|██████████████████████████████████████▊| 7332/7360 [18:14<00:03,  8.20it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
100%|██████████████████████████████████████▊| 7333/7360 [18:15<00:06,  4.16it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
100%|██████████████████████████████████████▊| 7334/7360 [18:15<00:05,  4.87it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
100%|██████████████████████████████████████▊| 7335/7360

100%|██████████████████████████████████████▉| 7358/7360 [18:18<00:00,  6.12it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
100%|██████████████████████████████████████▉| 7359/7360 [18:18<00:00,  6.65it/s]/var/folders/d5/_lh0npv17cnfv_j03v976_kr0000gn/T/ipykernel_8217/1329571255.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(aux)
100%|███████████████████████████████████████| 7360/7360 [18:19<00:00,  6.70it/s]
